In [116]:
import qibo
import torch
import numpy as np
from qibo import Circuit, gates, models
from qibo.optimizers import sgd

from qibochem.driver.molecule import Molecule
from qibochem.ansatz import hf_circuit, ucc_circuit
from qibochem.measurement import expectation, expectation_from_samples
from qibochem.ansatz import he_circuit

In [8]:
qibo.set_backend("numpy")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bohr_angs = 0.529177210903

[Qibo 0.2.9|INFO|2024-10-14 21:16:26]: Using numpy backend on /CPU:0


In [62]:
def h2(x):
    molecule = Molecule([('H', (0, 0, 0)), ('H', (0, 0, x))])
    molecule.run_pyscf()
    return molecule

In [63]:
molecule = h2(0.7)
hamiltonian = molecule.hamiltonian()

converged SCF energy = -1.11734903499028


In [5]:
nlayers = 1
nqubits = molecule.nso # Number of molecular spin-orbitals considered for the molecule
ntheta = 2 * nqubits * nlayers
circuit = he_circuit(nqubits, nlayers)
print(circuit.draw())

q0: ─RY─RZ─o─────Z─
q1: ─RY─RZ─Z─o───|─
q2: ─RY─RZ───Z─o─|─
q3: ─RY─RZ─────Z─o─


In [94]:
vqe = models.VQE(circuit, hamiltonian)
best, params, extra = vqe.minimize(np.random.uniform(0, 2 * np.pi, ntheta))
best, params

(-1.1173490349878505,
 array([ 3.14159263e+00,  6.50430197e+00,  3.14159669e+00,  1.71830858e+00,
         6.28318531e+00,  3.20931093e+00, -1.17805883e-10,  4.74157488e+00]))

In [101]:
def cost(params, x):
    molecule = h2(x)
    hamiltonian = molecule.hamiltonian()

    circuit.set_parameters(params)
    return expectation_from_samples(circuit, hamiltonian, n_shots=8096)

In [143]:
theta = torch.randn(ntheta, dtype=torch.float32, requires_grad=True, device=device)
x = torch.tensor([0.725], dtype=torch.float32, requires_grad=True, device=device)

learning_rate_theta = 0.001
learning_rate_x = 0.5

optimizer_theta = torch.optim.SGD([theta], lr=learning_rate_theta)
optimizer_x = torch.optim.SGD([x], lr=learning_rate_x)

In [144]:
def finite_diff(f, x, delta=0.01):
    """Compute the central-difference finite difference of a function"""
    gradient = []

    for i in range(len(x)):
        shift = np.zeros_like(x)
        shift[i] += 0.5 * delta
        res = (f(x + shift) - f(x - shift)) * delta**-1
        gradient.append(res)

    return gradient


def grad_x(params, x):
    grad_h = finite_diff(lambda x: h2(x).hamiltonian(), x)
    circuit.set_parameters(params)
    grad = [expectation_from_samples(circuit, obs, n_shots=1024) for obs in grad_h]
    return np.array(grad)

In [145]:
energy = []

# store the values of the bond length
bond_length = []

for n in range(100):
    # -------------------------
    # Optimize the circuit parameters (theta)
    # -------------------------
    optimizer_theta.zero_grad()
    energy_theta = cost(theta.detach().numpy(), x.detach().numpy()[0])
    energy_tensor = torch.tensor(energy_theta, requires_grad=True)
    energy_tensor.backward()
    optimizer_theta.step()

    # -------------------------
    # Optimize the nuclear coordinates (x)
    # -------------------------
    optimizer_x.zero_grad()
    energy_x = cost(theta.detach().numpy(), x.detach().numpy()[0])
    energy_tensor = torch.tensor(energy_x, requires_grad=True)
    energy_tensor.backward()
    optimizer_x.step()

    # Record energy and bond length
    with torch.no_grad():
        current_energy = cost(theta.detach().numpy(), x.detach().numpy()[0])
        energy.append(current_energy)
        current_bond_length = x.item()
        bond_length.append(current_bond_length)

    # Logging every 4 steps
    if n % 4 == 0:
        print(f"Step = {n},  E = {current_energy:.8f} Ha,  bond length = {current_bond_length:.5f} A")


converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
Step = 0,  E = -0.60618206 Ha,  bond length = 0.72500 A
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
Step = 4,  E = -0.60408325 Ha,  bond length = 0.72500 A
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.11734326912258
converged SCF energy = -1.1

In [146]:
energy

[-0.6061820633155466,
 -0.5962958030790737,
 -0.5964665056068887,
 -0.6016064822991345,
 -0.6040832514344299,
 -0.6080083614739347,
 -0.5992682155794824,
 -0.6065879012508939,
 -0.6036342806715533,
 -0.6008581979803498,
 -0.5989052984041017,
 -0.6037107145258342,
 -0.6025012197489741,
 -0.6043166203386421,
 -0.6087082223893998,
 -0.6047376353940945,
 -0.6027006051147257,
 -0.6086406895328667,
 -0.6064737771114184,
 -0.6026061269676661,
 -0.5988227253444531,
 -0.6062065900940339,
 -0.6038228425940226,
 -0.6055486881154306,
 -0.6083009156933092,
 -0.6064613692820886,
 -0.6016909801355008,
 -0.6013230305416466,
 -0.6101945991791777,
 -0.5961040554455432,
 -0.5955938972538968,
 -0.6045651969526169,
 -0.6035865424781304,
 -0.5999677584592262,
 -0.6074730788642904,
 -0.6011802579228336,
 -0.6169005487009245,
 -0.6088850863535424,
 -0.6047932681895345,
 -0.6018440439171628,
 -0.605914868860435,
 -0.6124641897581722,
 -0.5965235043784951,
 -0.6057578588151233,
 -0.5965621606200449,
 -0.6062515